In [2]:
import requests
base_url = 'http://localhost:2000'

In [3]:
class Session(requests.Session):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.headers.update({'Content-Type': 'application/json'})
    
    def get(self, url, *args, **kwargs):
        return super().get(base_url + url, *args, **kwargs)
    
    def post(self, url, *args, **kwargs):
        return super().post(base_url + url, *args, **kwargs)
    
    def patch(self, url, *args, **kwargs):
        return super().patch(base_url + url, *args, **kwargs)
    
    def delete(self, url, *args, **kwargs):
        return super().delete(base_url + url, *args, **kwargs)

client = Session()
resp = client.get("/ping")
resp.json()

{'data': 'pong!'}

In [9]:
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

/Users/aaryanpagar/Library/Python/3.11/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [53]:
query = "Nip IND ETF Junior BeES".upper()
print(query)


docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()
# doc = await db.collection("tickers").where("ticker", "<=", query).where("ticker", ">=", query + "~").get()

if docs:
    print(docs[0].to_dict())

print(docs)

NIP IND ETF JUNIOR BEES
{'created_at': '2024-05-05T12:20:40.104545Z', 'exchange': 'NSE', 'lot_size': 1, 'upstox_instrument_key': 'NSE_EQ|INF732E01045', 'exchange_token': 10939, 'updated_at': '2024-05-05T12:20:40.104547Z', 'ticker': 'JUNIORBEES', 'instrument_type': 'EQUITY', 'name': 'NIP IND ETF JUNIOR BEES'}


/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_8262/3214706254.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()


In [61]:
from datetime import datetime
date = datetime.now()
print(date.strftime("%Y-%m-%d"))

2024-05-05


In [62]:
def format_date_path(date: datetime) -> str:
    return date.strftime("%Y-%m-%d")

In [5]:
from datetime import datetime
from typing import Dict, Optional, Annotated
from pydantic import BaseModel, Field, validator, StringConstraints

def format_date_path(date: datetime) -> str:
    try:
        return date.strftime("%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")
        
def validate_date_path(date: str):
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")

class CandleBase(BaseModel):
    open: float
    high: float
    low: float
    close: float
    volume: int
    open_interest: Optional[int] = None

class MonthlyCandleBase(BaseModel):
    month: Annotated[str, StringConstraints(pattern=r"^(0[1-9]|1[0-2])$")]
    year: Annotated[str, StringConstraints(pattern=r"^(19\d{2}|20\d{2})$")]
    daily: Dict[str, CandleBase] = Field(default_factory=dict)
    monthly: Optional[CandleBase] = None

    @validator("daily", pre=True, allow_reuse=True)
    def validate_daily(cls, v):
        for key in v:
            validate_date_path(key)
        return v

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_52500/2471255407.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator("daily", pre=True, allow_reuse=True)


In [29]:
ticker = "100"
date_from = "2015-01-21"
date_to = "2016-01-20"
params = {"start_date": date_from, "end_date": date_to}
resp = client.get(f"/tickers/{ticker}/candles", params=params)

resp.json()

{'2016-01-01': {'open': 864.55,
  'high': 871.0,
  'low': 860.2,
  'close': 867.45,
  'volume': 175063,
  'open_interest': 0},
 '2016-01-04': {'open': 868.05,
  'high': 879.0,
  'low': 865.0,
  'close': 870.6,
  'volume': 274274,
  'open_interest': 0},
 '2016-01-05': {'open': 871.0,
  'high': 878.0,
  'low': 868.2,
  'close': 871.0,
  'volume': 442613,
  'open_interest': 0},
 '2016-01-06': {'open': 871.7,
  'high': 884.7,
  'low': 855.1,
  'close': 861.8,
  'volume': 251918,
  'open_interest': 0},
 '2016-01-07': {'open': 857.3,
  'high': 859.5,
  'low': 835.2,
  'close': 838.25,
  'volume': 180042,
  'open_interest': 0},
 '2016-01-08': {'open': 846.0,
  'high': 852.75,
  'low': 825.3,
  'close': 845.35,
  'volume': 210505,
  'open_interest': 0},
 '2016-01-11': {'open': 840.0,
  'high': 845.9,
  'low': 828.9,
  'close': 841.25,
  'volume': 126301,
  'open_interest': 0},
 '2016-01-12': {'open': 841.85,
  'high': 851.8,
  'low': 831.85,
  'close': 834.8,
  'volume': 124804,
  'open_intere

In [4]:
date_from = "2024-01-01"
date_to = "2024-02-02"
date_from <= date_to

True

In [11]:
date = "2024-07-02"
id = "SCAW_0001"
start_date = "2016-01-01"
end_date = "2024-05-20"
params = {"start_date": start_date, "end_date": end_date}

resp = client.get(f"/smallcases/{id}/indexes", params=params)
resp.json()

{'2018-07-17': {'smallcase': 100.0,
  'benchmark': 100.0,
  'kelly': None,
  'rebalance_occured': True},
 '2018-07-18': {'smallcase': 99.33,
  'benchmark': 99.63,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-19': {'smallcase': 99.29,
  'benchmark': 99.37,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-20': {'smallcase': 99.52,
  'benchmark': 99.87,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-23': {'smallcase': 100.22,
  'benchmark': 100.68,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-24': {'smallcase': 100.5,
  'benchmark': 101.41,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-25': {'smallcase': 100.55,
  'benchmark': 101.38,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-26': {'smallcase': 100.63,
  'benchmark': 101.78,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-27': {'smallcase': 100.77,
  'benchmark': 102.77,
  'kelly': None,
  'rebalance_occured': False},
 '2018-07-30': {'smallcase': 101.1,
  

In [13]:
from google.cloud import firestore

q = db.collection("smallcases/SCAW_0001/constituents").order_by("__name__").limit(1)

docs = await q.get()

for doc in docs:
    print(doc.id)

2018-07
